# How does the literary and language structure of Shakespearean text influence the effectiveness of an LLM to generate authentic Shakespearean text?

Can it generate text that resembles Shakespeares original works by picking up on typical Shakespearan themes, language, and flow?

In [ ]:
import numpy as np
import tensorflow as tf
import os
import time

In [ ]:
# Load Shakespeare text

path_to_file = "train_shakespeare.txt"

# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

Length of text: 521800 characters
77 unique characters


In [ ]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

In [ ]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                             return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [ ]:
# Function to generate text
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 100000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

In [ ]:
dataset = sequences.map(split_input_target)

# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [ ]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

# Checkpoint directory
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

# Compiling the model
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# Training the model
EPOCHS = 10

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/10
29/80 [=========>....................] - ETA: 4:26 - loss: 3.4327

KeyboardInterrupt: 

In [ ]:
# Restore the latest checkpoint
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
model.build(tf.TensorShape([1, None]))

# Generate text starting from 'ROMEO: '
print(generate_text('training_checkpoints/ckpt_30.data-00000-of-00001', start_string=u"ROMEO: "))


AttributeError: 'str' object has no attribute 'reset_states'

In [ ]:
import numpy as np
import tensorflow as tf
import os
import time

# Check if GPU is available
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

# Load Shakespeare text
path_to_file = "train_shakespeare.txt"

# Read, then decode for py2 compat.
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')

# Length of text is the number of characters in it
print('Length of text: {} characters'.format(len(text)))

# The unique characters in the file
vocab = sorted(set(text))
print('{} unique characters'.format(len(vocab)))

# Creating a mapping from unique characters to indices
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

text_as_int = np.array([char2idx[c] for c in text])

# The maximum length sentence we want for a single input in characters
seq_length = 100
examples_per_epoch = len(text)//(seq_length+1)

# Create training examples / targets
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

sequences = char_dataset.batch(seq_length+1, drop_remainder=True)

def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.LSTM(rnn_units,
                             return_sequences=True,
                             stateful=True,
                             recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

# Function to generate text
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 10000

    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ''.join(text_generated))

dataset = sequences.map(split_input_target)

# Batch size
BATCH_SIZE = 32

# Buffer size to shuffle the dataset
BUFFER_SIZE = 100000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

# Check if GPU is available
if len(tf.config.experimental.list_physical_devices('GPU')) > 0:
    print("GPU is available. Training on GPU...")
    with tf.device('/GPU:0'):
        model = build_model(
            vocab_size=len(vocab),
            embedding_dim=embedding_dim,
            rnn_units=rnn_units,
            batch_size=BATCH_SIZE)

        # Checkpoint directory
        checkpoint_dir = './training_checkpoints'
        checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

        checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
            filepath=checkpoint_prefix,
            save_weights_only=True)

        # Compiling the model
        model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

        # Training the model
        EPOCHS = 200

        history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

        # Restore the latest checkpoint
        model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
        model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
        model.build(tf.TensorShape([1, None]))

        # Generate text starting from 'ROMEO: '
        print(generate_text(model, start_string=u"ROMEO: "))
else:
    print("GPU not available. Training on CPU...")
    model = build_model(
        vocab_size=len(vocab),
        embedding_dim=embedding_dim,
        rnn_units=rnn_units,
        batch_size=BATCH_SIZE)

    # Checkpoint directory
    checkpoint_dir = './training_checkpoints'
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix,
        save_weights_only=True)

    # Compiling the model
    model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

    # Training the model
    EPOCHS = 100

    history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

    # Restore the latest checkpoint
    model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
    model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))
    model.build(tf.TensorShape([1, None]))

    # Generate text starting from 'ROMEO: '
    print(generate_text(model, start_string=u"ROMEO: "))


Num GPUs Available:  1
Length of text: 521800 characters
77 unique characters
GPU is available. Training on GPU...
Epoch 1/200


161/161 [==============================] - 5s 19ms/step - loss: 2.6765
Epoch 2/200
161/161 [==============================] - 3s 15ms/step - loss: 1.9725
Epoch 3/200
161/161 [==============================] - 3s 15ms/step - loss: 1.7270
Epoch 4/200
161/161 [==============================] - 3s 14ms/step - loss: 1.5846
Epoch 5/200
161/161 [==============================] - 3s 14ms/step - loss: 1.4903
Epoch 6/200
161/161 [==============================] - 3s 14ms/step - loss: 1.4180
Epoch 7/200
161/161 [==============================] - 3s 14ms/step - loss: 1.3585
Epoch 8/200
161/161 [==============================] - 3s 14ms/step - loss: 1.3037
Epoch 9/200
161/161 [==============================] - 3s 14ms/step - loss: 1.2494
Epoch 10/200
161/161 [==============================] - 3s 14ms/step - loss: 1.1953
Epoch 11/200
161/161 [==============================] - 3s 14ms/step - loss: 1.1378
Epoch 12/200
161/161 [==============================] - 3s 14ms/step - loss: 1.0777
Epoch 13/200


# **Analysis**

**Language and Vocabulary:** Shakespeare is known for his extensive vocabulary and creative use of language. The generated text includes words and phrases that seem to fit the Shakespeare style such as Ahonord, passado, and bellows-mender, which evoke the language of Shakespearean plays.
Rhythm and Meter: Shakespeare often wrote in iambic pentameter, a rhythmic pattern of stressed and unstressed syllables. While the generated text doesn't exactly showcase this meter, it contains rhythmic elements similar to that of Shakespeare's work.


**Characters and Themes:** The text contains classic characters from the works of Shakespeare and they all seem to have the same character traits and personalities as they do in real Shakespeare. Characters such as Romeo, Hamlet, and Macbeth constantly mention themes of death, love, power, or make sly sexual innuendos.

    “in the morning’s eyeing blessed my love’s tongue, bring him silently”

    “Here’s my father and rest, but all is death, but body’s bones.”

    “I will not stay in the siege of loving death.
    But let the mind to you; farewell. Now I am alone.”

    “They bleed so that I may love thee.”

    “And Tybalt’s dead”

    “Sir, in my heart I am an oppressor”

    “Come, sir, There’s blood upon your brow”

    “I should live to be born. my lips that lie look upon within.”

    “I had most need? The Queen and his mouth, his finespire, abound for if I can see”

**Dramatic Elements:** Shakespearean plays are known for their dramatic aspects. While not fully included in this, the generated text contains elements of drama such as fights. One spot I particularly enjoyed because it made me chuckle is when a fight spontaneously broke out.

    NURSE:
    She sees, my lord, and I will wear it.

    HAMLET:
    I am but merry!—
    I’ll call up our wisest fair Jepantasion.

    POLONIUS:
    Have I, And put it to you.

    ROMEO:
    Come, sir, your passado.

    [They fight.]

Not only did I find this part particularly funny, but I think it also showcases some of the spontaneity of Shakespeare's writing, as sometimes when reading Shakespeare you’re on the edge of your seat, not always knowing what to expect next.

**In summary**, while the generated text may not achieve the exact mastery and skill of Shakespeare's writing, it captures enough elements of his style to evoke a sense of familiarity and with his works. In my opinion this highlights Shakespeare's uniqueness and influence of his works, which continue to inspire and influence writers centuries later.

**Honorable mentions** of lines that made me laugh

    “So let him dumb head”

    “FIRST CLOWN:
    [Sings.]
    An old maid’s, or their own distracted groves,
    The virtue of the thure, Lords, Or if thou know’st mine eyes,
    And braggart with my tongue!—But, gentle sweet, you shall hear, go join you, I’ll fa you. Do you notice men?,
    I qual night, and left me with rum to say, I saw the other senses,
    Or my true knight!”

**Please note this results were from my specific generation, your generations may produce varying results**